In [9]:
import itertools
from collections import defaultdict
import networkx as nx
import math

In [ ]:
def euclidean(p1, p2):
    return math.dist(p1, p2)

def extract_candidate_distances(clients1, clients2, facilities1, facilities2, distance_fn):
    distances = set()
    for j in clients1 + clients2:
        for i in facilities1 + facilities2:
            distances.add(distance_fn(j, i))
    return sorted(distances)

def greedy_k_supplier(clients, facilities, k, R, distance_fn):
    """
    Greedy algorithm: Place centers such that all clients are within R.
    Each new facility covers as many uncovered clients as possible.
    """
    uncovered = set(clients)
    used_facilities = set()

    while uncovered and len(used_facilities) < k:
        # For each facility, count how many uncovered clients it can cover
        best_f, covered = None, set()
        for f in facilities:
            covered_now = {j for j in uncovered if distance_fn(j, f) <= R}
            if len(covered_now) > len(covered):
                best_f, covered = f, covered_now

        if not covered:
            return None  # Can't cover remaining clients with current R

        used_facilities.add(best_f)
        uncovered -= covered

    if uncovered:
        return None #cant satisfy all clients with radius r
    return used_facilities

def build_bipartite_graph(facilities1, facilities2, B, distance_fn):
    G = nx.Graph()
    for i, f1 in enumerate(facilities1):
        for j, f2 in enumerate(facilities2):
            if distance_fn(f1, f2) <= B:
                G.add_edge(f"f1_{i}", f"f2_{j}")
    return G

'''
1. len different ho sakta hai set ka
2. len < k ho sakta hai set ka
3. final maximum perfect mapping edges = min(len(left), len(right))
'''

def has_valid_matching(G, k):
    if G.number_of_edges() == 0:
        return False
    

    ## modify
    try:
        matching = nx.bipartite.maximum_matching(G)
        return len(matching) // 2 >= k
    except nx.NetworkXPointlessConcept:
        return False

def is_feasible(R, B, clients1, clients2, facilities1, facilities2, k, distance_fn=euclidean):
    S1 = greedy_k_supplier(clients1, facilities1, k, R, distance_fn)
    S2 = greedy_k_supplier(clients2, facilities2, k, R, distance_fn)
    if S1 is None or S2 is None:
        return False

    G = build_bipartite_graph(list(S1), list(S2), B, distance_fn)
    return has_valid_matching(G, k)

def binary_search_R_star(clients1, clients2, facilities1, facilities2, B, k, distance_fn=euclidean):
    candidates = extract_candidate_distances(clients1, clients2, facilities1, facilities2, distance_fn)
    low, high = 0, len(candidates) - 1
    best_R = None

    while low <= high:
        mid = (low + high) // 2
        R = candidates[mid]
        if is_feasible(R, B, clients1, clients2, facilities1, facilities2, k, distance_fn):
            best_R = R
            high = mid - 1
        else:
            low = mid + 1

    return best_R


In [13]:
clients1 = [(0, 0), (1, 1)]
clients2 = [(5, 5), (6, 6)]
facilities1 = [(0, 1), (1, 0)]
facilities2 = [(5, 6), (6, 5)]
B = 2  # movement bound
k = 2  # number of facilities

R_star = binary_search_R_star(clients1, clients2, facilities1, facilities2, B, k)
print("Guessed R*:", R_star)


Guessed R*: None
